In [1]:


from sklearn.metrics import accuracy_score, f1_score, precision_score
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np

from ucimlrepo import fetch_ucirepo


In [2]:
heart_disease = fetch_ucirepo(id=45)

X = heart_disease.data.features
Y = heart_disease.data.targets

Y = Y['num'].replace([1, 2, 3, 4], 1)

X['num'] = Y

# median = X['ca'].median()
# X['ca'].fillna(median, inplace=True)
# mode_category = X['thal'].mode()[0]
# X['thal'].fillna(mode_category, inplace=True)

X = X.dropna()

Wypełnianie brakujących wartości W kolumnie 'ca' oraz 'thal' wartością najczęściej występującą w tej kolumnie, powoduje gorsze wyniki modelu domyślnego, dlatego postanowiono, że usuniemy te próbki z danych.

In [3]:
def one_hot_encode(df, column, column_names):
    dummies = pd.get_dummies(df[column], prefix=column)
    column_names = [column + '_' + str(name) for name in column_names]
    dummies.columns = column_names
    dummies = dummies.astype('int64')
    df = pd.concat([df, dummies], axis=1)
    df.drop(column, axis=1, inplace=True)
    return df

In [4]:
X =  one_hot_encode(X, 'cp', ['typical_angina', 'atypical_angina', 'non-anginal_pain', 'asymptomatic'])
X = one_hot_encode(X, 'thal', ['normal', 'ST-T_wave_abnormality', 'left_ventricular_hypertrophy'])
X = one_hot_encode(X, 'slope', ['upsloping', 'flat', 'downsloping'])
X = one_hot_encode(X, 'restecg', ['normal', 'fixed_defect', 'reversable_defect'])

In [5]:
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 297 entries, 0 to 301
Data columns (total 23 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   age                                297 non-null    int64  
 1   sex                                297 non-null    int64  
 2   trestbps                           297 non-null    int64  
 3   chol                               297 non-null    int64  
 4   fbs                                297 non-null    int64  
 5   thalach                            297 non-null    int64  
 6   exang                              297 non-null    int64  
 7   oldpeak                            297 non-null    float64
 8   ca                                 297 non-null    float64
 9   num                                297 non-null    int64  
 10  cp_typical_angina                  297 non-null    int64  
 11  cp_atypical_angina                 297 non-null    int64  
 12 

In [6]:
# Extract target without missing values
Y = X['num']
X = X.drop('num', axis=1)

# Normalize data to [0, 1] range
X_norm = (X - X.min()) / (X.max() - X.min())

In [12]:
class Neuron():
    def __init__(self, num_of_weights, stand_dev=1):
        self.weights = np.random.normal(scale=stand_dev, size=num_of_weights)
        self.bias = np.random.normal(scale=stand_dev)
        self.X = None
        self.derivative = None

    @staticmethod
    def sigmoid(X):
        # return np.exp(-np.logaddexp(0, -X))
        return 1 / (1 + np.exp(-X))

    def sigmoid_derivative(self, X):
        return self.sigmoid(X) * (1 - self.sigmoid(X))

    def forward(self, inputs):
        self.X = inputs
        return self.sigmoid(np.dot(inputs, self.weights) + self.bias)

    def backward(self, error, weights_next_layer=None):
        if weights_next_layer is not None:
            error = error.T @ weights_next_layer
        self.derivative = error * self.sigmoid_derivative(np.dot(self.X,self.weights) + self.bias)
        return self.derivative
    
    def update(self, learning_rate):
        self.weights -=  learning_rate * np.dot(self.X.T, self.derivative)
        self.bias -= learning_rate * np.sum(self.derivative)
        self.X = None
        self.derivative = None

class NeuralNetwork():
    def __init__(self, num_of_inputs, hidden_layers, num_of_outputs, stand_dev=1.0):
        self.layers = []
        self.num_of_inputs = num_of_inputs
        self.hidden_layers = hidden_layers
        self.num_of_outputs = num_of_outputs
        self.num_of_hidden_layers = len(hidden_layers)
        self.stand_dev = stand_dev
        
        self.layers.append([Neuron(num_of_inputs, self.stand_dev) for _ in range(hidden_layers[0])])
        for i in range(1, self.num_of_hidden_layers):
            self.layers.append([Neuron(hidden_layers[i-1], self.stand_dev) for _ in range(hidden_layers[i])])
        if self.num_of_outputs == 1:
            self.layers.append([Neuron(hidden_layers[-1], self.stand_dev)])

    def cross_entropy(self, y, y_pred):
        return -np.sum(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred), axis=1)

    def cross_entropy_derivative(self, y, y_pred):
        return -np.sum(y / y_pred - (1 - y) / (1 - y_pred), axis=1)

    def forward(self, X):
        for layer in self.layers:
            X = np.array([neuron.forward(X) for neuron in layer]).T
        return X        

    def backward(self, output_error):
        # Iterate over the hidden layers in reverse order and calculate their gradients
        output_layer = self.layers[-1]
        output_error = np.array([neuron.backward(output_error) for neuron in output_layer])
        weights_next_layer = np.array([neuron.weights for neuron in output_layer]).T

        for layer in reversed(self.layers[:-1]):
            output_error = np.array([neuron.backward(output_error, weights_next_layer[index]) for index, neuron in enumerate(layer)])
            weights_next_layer = np.array([neuron.weights for neuron in layer]).T
        
        # Return the final gradients
        return output_error

    def update(self, learning_rate):
        for layer in self.layers:
            for neuron in layer:
                neuron.update(learning_rate)

    def fit(self, X, y, epochs=1000, batch_size=10, learning_rate=0.005):
        n_samples, n_features = X.shape
        # Jeśli batch_size jest większy niż liczba próbek, to ustawiamy go na liczbę próbek -> zwyczajnie uczymy na całym zbiorze GD
        if batch_size > n_samples:
            batch_size = n_samples
        cost_list = []
        for epoch in range(epochs):
            random_order = np.random.permutation(n_samples)
            X_shuffled = X.values[random_order]
            y_shuffled = y.values[random_order]
            # lista kosztów w paczkach dla każdej epoki
            cost_list_in_batch = []

            for batch_index in range(0, n_samples, batch_size):

                X_batch = X_shuffled[batch_index:batch_index + batch_size]
                y_batch = y_shuffled[batch_index:batch_index + batch_size].reshape(-1, 1)
                predictions = self.forward(X_batch)
                output_error = self.cross_entropy_derivative(y_batch, predictions)
                self.backward(output_error)
                self.update(learning_rate)
                
            cost = self.cross_entropy(y_batch, predictions)
            cost_list.append(np.mean(cost))

            if epoch % (epochs // 8) == 0:
                print(f'Epoch: {epoch}, Loss: {np.mean(cost)}')

            #jeśli koszt jest mniejszy niż epsilon to stop
            # if(i > 0 and abs(cost - self.cost_list[-1]) < self.epsilon ):
                # break
            # self.cost_list.append(cost)
            # iteration_cost = np.mean(cost_list_in_batch)
            # self.mean_cost_list.append(iteration_cost)
            # self.epoch_list.append(i)
        print(f'Epoch: {epoch}, Loss: {np.mean(cost)}')
        return self, cost_list


    def predict(self, X):
        return self.forward(X)

In [13]:
def sigmoid(X):
    return np.exp(-np.logaddexp(0, -X))

def sigmoid_derivative(X):
    return sigmoid(X) * (1 - sigmoid(X))

def cross_entropy(y, y_pred):
    return -np.sum(y * np.log(y_pred) + (1 - y) * np.log(1 - y_pred), axis=1)

def cross_entropy_derivative(y, y_pred):
    return -np.sum(y / y_pred - (1 - y) / (1 - y_pred), axis=1)

class Neuron():
    def __init__(self, num_of_weights, activation_function, activation_derivative, stand_dev=1):
        self.weights = np.random.normal(scale=stand_dev, size=num_of_weights)
        self.bias = np.random.normal(scale=stand_dev)
        self.X = None
        self.activation_function = activation_function
        self.activation_derivative = activation_derivative

    def forward(self, inputs):
        self.X = inputs
        return self.activation_function(np.dot(inputs, self.weights) + self.bias)

    def backward(self, error, weights_next_layer=None):
        if weights_next_layer is not None:
            error = error.T @ weights_next_layer
        self.derivative = error * self.activation_derivative(np.dot(self.X, self.weights) + self.bias)
        return self.derivative

    def update(self, learning_rate):
        self.weights -= learning_rate * np.dot(self.X.T, self.derivative)
        self.bias -= learning_rate * np.sum(self.derivative)
        self.X = None
        self.derivative = None



class NeuralNetwork():
    def __init__(self, num_of_inputs, hidden_layers, num_of_outputs, activation_function = sigmoid, activation_derivative = sigmoid_derivative, loss_function = cross_entropy, loss_derivative = cross_entropy_derivative, stand_dev=1.0):
        self.layers = []
        self.num_of_inputs = num_of_inputs
        self.hidden_layers = hidden_layers
        self.num_of_outputs = num_of_outputs
        self.num_of_hidden_layers = len(hidden_layers)
        self.activation_function = activation_function
        self.activation_derivative = activation_derivative
        self.loss_function = loss_function
        self.loss_derivative = loss_derivative
        self.stand_dev = stand_dev

        self.layers.append([Neuron(num_of_inputs, activation_function, activation_derivative, self.stand_dev) for _ in range(hidden_layers[0])])
        for i in range(1, self.num_of_hidden_layers):
            self.layers.append([Neuron(hidden_layers[i-1], activation_function, activation_derivative, self.stand_dev) for _ in range(hidden_layers[i])])
        if self.num_of_outputs == 1:
            self.layers.append([Neuron(hidden_layers[-1], activation_function, activation_derivative, self.stand_dev)])

    def forward(self, X):
        for layer in self.layers:
            X = np.array([neuron.forward(X) for neuron in layer]).T
        return X

    def backward(self, output_error):
        output_layer = self.layers[-1]
        output_error = np.array([neuron.backward(output_error) for neuron in output_layer])
        weights_next_layer = np.array([neuron.weights for neuron in output_layer]).T

        for layer in reversed(self.layers[:-1]):
            output_error = np.array([neuron.backward(output_error, weights_next_layer[index]) for index, neuron in enumerate(layer)])
            weights_next_layer = np.array([neuron.weights for neuron in layer]).T

        return output_error

    def update(self, learning_rate):
        for layer in self.layers:
            for neuron in layer:
                neuron.update(learning_rate)

    def fit(self, X, y, epochs=1000, batch_size=10, learning_rate=0.005):
        n_samples, n_features = X.shape
        if batch_size > n_samples:
            batch_size = n_samples
        cost_list = []

        for epoch in range(epochs):
            random_order = np.random.permutation(n_samples)
            X_shuffled = X.values[random_order]
            y_shuffled = y.values[random_order]

            cost_list_in_batch = []

            for batch_index in range(0, n_samples, batch_size):
                X_batch = X_shuffled[batch_index:batch_index + batch_size]
                y_batch = y_shuffled[batch_index:batch_index + batch_size].reshape(-1, 1)
                predictions = self.forward(X_batch)
                output_error = self.loss_derivative(y_batch, predictions)
                self.backward(output_error)
                self.update(learning_rate)

            cost = self.loss_function(y_batch, predictions)
            cost_list.append(np.mean(cost))

            if epoch % (epochs // 8) == 0:
                print(f'Epoch: {epoch}, Loss: {np.mean(cost)}')

        print(f'Epoch: {epoch}, Loss: {np.mean(cost)}')
        return self, cost_list

    def predict(self, X):
        return self.forward(X)




Klasa Neuron reprezentuje pojedynczy neuron w sieci neuronowej. Przyjmuje następujące parametry podczas inicjalizacji:

- num_of_weights: Liczba wag (parametrów) neuronu.
- activation_function: Funkcja aktywacji neuronu.
- activation_derivative: Pochodna funkcji aktywacji neuronu.
- stand_dev: Odchylenie standardowe używane do inicjalizacji wag neuronu (domyślnie 1).
Metody klasy Neuron obejmują:

- forward(self, inputs): Wykonuje krok przód, obliczając wyjście neuronu na podstawie wejścia i funkcji aktywacji.
- backward(self, error, weights_next_layer=None): Wykonuje krok wstecz, obliczając pochodną błędu względem wejścia neuronu. Jeśli - dostępne są wagi warstwy następnej, można je przekazać, aby obliczyć błąd wsteczny w oparciu o te wagi.
- update(self, learning_rate): Aktualizuje wagi neuronu na podstawie pochodnej błędu i współczynnika uczenia.

In [14]:
seed = 42
np.random.seed(seed)
X_train, X_test, Y_train, Y_test = train_test_split(X_norm, Y, test_size=0.2)
mlp = NeuralNetwork(num_of_inputs=X_train.shape[1], hidden_layers=[8, 4], num_of_outputs=1)
mlp.fit(X_train, Y_train, epochs=1000, batch_size=10, learning_rate=0.005)
predictions = mlp.predict(X_test)
predictions = np.where(predictions > 0.5, 1, 0)
print(f'Accuracy: {accuracy_score(Y_test, predictions)}')
print(f'Precision: {precision_score(Y_test, predictions)}')
print(f'F1: {f1_score(Y_test, predictions)}')


Epoch: 0, Loss: 0.8366587713720809
Epoch: 125, Loss: 0.17555607988440508
Epoch: 250, Loss: 0.155472601691947
Epoch: 375, Loss: 0.19044257397183223
Epoch: 500, Loss: 0.08761761573339792
Epoch: 625, Loss: 0.26155733617304106
Epoch: 750, Loss: 0.02290420071616795
Epoch: 875, Loss: 0.030357829459238338
Epoch: 999, Loss: 0.11920780047209564
Accuracy: 0.85
Precision: 0.8
F1: 0.816326530612245


In [12]:
dim_hidd_1 = [5, 5]
dim_hidd_1_nn = NeuralNetwork(X_train.shape[1], [8,4], 1)
trained, lossh =  dim_hidd_1_nn.fit(X_train, Y_train, epochs=1000, batch_size=10, learning_rate=0.005)

predictions = dim_hidd_1_nn.predict(X_test)
predictions = np.where(predictions > 0.5, 1, 0)
print(f'Accuracy: {accuracy_score(Y_test, predictions)}')
print(f'Precision: {precision_score(Y_test, predictions)}')
print(f'F1: {f1_score(Y_test, predictions)}')

Epoch: 0, Loss: 0.6694471193319574
Epoch: 125, Loss: 0.38844936451380113
Epoch: 250, Loss: 0.26240670050851156
Epoch: 375, Loss: 0.12076295494995304
Epoch: 500, Loss: 0.04224397525743791
Epoch: 625, Loss: 0.13869862523886195
Epoch: 750, Loss: 0.2134002784342621
Epoch: 875, Loss: 0.11552206226093573
Epoch: 999, Loss: 0.5652591722247716
Accuracy: 0.85
Precision: 0.8
F1: 0.816326530612245


In [26]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=seed)
X_train_n, X_test_n, Y_train_n, Y_test_n = train_test_split(X_norm, Y, test_size=0.2, random_state=seed)

# domyślna sieć
default_hidden_layers = [10, 5]

#1.Różnej wymiarowości warstwy ukrytej
dim_hidd_1 = [5, 5]
dim_hidd_2 = [20, 10]
dim_hidd_3 = [50, 25]
#2. Różnej wartości współczynnika uczenia
learning_rate_1 = 0.001
learning_rate_2 = 0.01
#3. Różnej wartości parametru standaryzacji
stand_dev_1 = 0.1
stand_dev_2 = 5
#4.danych znormalizownaych i nieznormalizowanych
unnorm = X_train, Y_train
#5. Różnej liczby watstw ukrytych
hidden_layers_size_1 = [10]
hidden_layers_size_2 = [10,5,2]
hidden_layers_size_3 = [10,5,3,2]

default_nn = NeuralNetwork(num_of_inputs=X_train.shape[1], hidden_layers=default_hidden_layers, num_of_outputs=1)
dim_hidd_1_nn = NeuralNetwork(num_of_inputs=X_train.shape[1], hidden_layers=dim_hidd_1, num_of_outputs=1)
dim_hidd_2_nn = NeuralNetwork(num_of_inputs=X_train.shape[1], hidden_layers=dim_hidd_2, num_of_outputs=1)
dim_hidd_3_nn = NeuralNetwork(num_of_inputs=X_train.shape[1], hidden_layers=dim_hidd_3, num_of_outputs=1)
learning_rate_1_nn = NeuralNetwork(num_of_inputs=X_train.shape[1], hidden_layers=default_hidden_layers, num_of_outputs=1)
learning_rate_2_nn = NeuralNetwork(num_of_inputs=X_train.shape[1], hidden_layers=default_hidden_layers, num_of_outputs=1)
stand_dev_1_nn = NeuralNetwork(num_of_inputs=X_train.shape[1], hidden_layers=default_hidden_layers, num_of_outputs=1, stand_dev=stand_dev_1)
stand_dev_2_nn = NeuralNetwork(num_of_inputs=X_train.shape[1], hidden_layers=default_hidden_layers, num_of_outputs=1, stand_dev=stand_dev_2)
unnorm_nn = NeuralNetwork(num_of_inputs=X_train.shape[1], hidden_layers=default_hidden_layers, num_of_outputs=1)
hidden_layers_size_1_nn = NeuralNetwork(num_of_inputs=X_train.shape[1], hidden_layers=hidden_layers_size_1, num_of_outputs=1)
hidden_layers_size_2_nn = NeuralNetwork(num_of_inputs=X_train.shape[1], hidden_layers=hidden_layers_size_2, num_of_outputs=1)

print('Default')
default_mlp, loss_history_1 = default_nn.fit(X_train, Y_train)
print('----------------')
print('Dim hidd 1')
dim_hidd_1_mlp, loss_history_2 = dim_hidd_1_nn.fit(X_train, Y_train)
print('----------------')
print('Dim hidd 2')
dim_hidd_2_mlp, loss_history_3 = dim_hidd_2_nn.fit(X_train, Y_train)
print('----------------')
print('Dim hidd 3')
dim_hidd_3_mlp, loss_history_4 = dim_hidd_3_nn.fit(X_train, Y_train)
print('----------------')
print('Learning rate 1')
learning_rate_1_mlp, loss_history_5 = learning_rate_1_nn.fit(X_train, Y_train, learning_rate=learning_rate_1)
print('----------------')
print('Learning rate 2')
learning_rate_2_mlp, loss_history_6 = learning_rate_2_nn.fit(X_train, Y_train, learning_rate=learning_rate_2)
print('----------------')
print('Stand dev 1')
stand_dev_1_mlp, loss_history_7 = stand_dev_1_nn.fit(X_train, Y_train)
print('----------------')
print('Stand dev 2')
stand_dev_2_mlp, loss_history_8 = stand_dev_2_nn.fit(X_train, Y_train)
print('----------------')
print('Unnorm')
unnorm_mlp, loss_history_9 = unnorm_nn.fit(X_train_n, Y_train_n)
print('----------------')
print('Hidden layers size 1')
hidden_layers_size_1_mlp, loss_history_10 = hidden_layers_size_1_nn.fit(X_train, Y_train)
print('----------------')
print('Hidden layers size 2')
hidden_layers_size_2_mlp, loss_history_11 = hidden_layers_size_2_nn.fit(X_train, Y_train)
print('----------------')


Default
Epoch: 0, Loss: 0.7524962040592994
Epoch: 125, Loss: 0.7009894242599631
Epoch: 250, Loss: 0.6878027957694791
Epoch: 375, Loss: 0.687521041446641
Epoch: 500, Loss: 0.6872562733784356
Epoch: 625, Loss: 0.6642521104997606
Epoch: 750, Loss: 0.6887148005676039
Epoch: 875, Loss: 0.6763290951050083
Epoch: 999, Loss: 0.700457546494633
----------------
Dim hidd 1
Epoch: 0, Loss: 0.7788148788470411


C:\Users\filip\AppData\Local\Temp\ipykernel_16188\2817502624.py:2: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-X))


Epoch: 125, Loss: 0.6884267189244628
Epoch: 250, Loss: 0.7141863191867582
Epoch: 375, Loss: 0.6865939279111491
Epoch: 500, Loss: 0.6748181432260546
Epoch: 625, Loss: 0.688133295998273
Epoch: 750, Loss: 0.6795714939763995
Epoch: 875, Loss: 0.7023511686459337
Epoch: 999, Loss: 0.7029005102136424
----------------
Dim hidd 2
Epoch: 0, Loss: 0.686054370092099
Epoch: 125, Loss: 0.6940996139792415
Epoch: 250, Loss: 0.7105935214175999
Epoch: 375, Loss: 0.7087787605726472
Epoch: 500, Loss: 0.6847954959929551
Epoch: 625, Loss: 0.7033544615807016
Epoch: 750, Loss: 0.7211842799646612
Epoch: 875, Loss: 0.6512631052658333
Epoch: 999, Loss: 0.7082211390154834
----------------
Dim hidd 3
Epoch: 0, Loss: 0.6898126656276018
Epoch: 125, Loss: 0.6554569368566225
Epoch: 250, Loss: 0.7493743587541
Epoch: 375, Loss: 0.6725021344334061
Epoch: 500, Loss: 0.6637303372042327
Epoch: 625, Loss: 0.8089405860988051
Epoch: 750, Loss: 0.7283553390593047
Epoch: 875, Loss: 0.6764177345470307
Epoch: 999, Loss: 0.66641668

In [27]:

predictions = default_mlp.predict(X_test)
predictions = np.where(predictions > 0.5, 1, 0)
print(f'Accuracy: {accuracy_score(Y_test, predictions)}')
print(f'Precision: {precision_score(Y_test, predictions)}')
print(f'F1: {f1_score(Y_test, predictions)}')




Accuracy: 0.6
Precision: 0.0
F1: 0.0


C:\Users\filip\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


XD